In [1]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

import pandas as pd

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
df_cian = pd.read_csv("./data/cian_houses_training.csv")
df_cian.head()

,price,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,year,...,house_type,heating,breakdown,accomodation_type,author,room_count,floor_ratio,room_ratio,living_ratio,kitchen_ratio
0,18374400,55.802530,37.620945,Марьина Роща,34,49,38.28,11.80,10.23,2027,...,Монолитно-кирпичный,unknown,unknown,Новостройка,КОРТРОС,1,0.693878,38.28,0.308255,0.267241
1,8170000,55.552637,37.337172,Аэропорт Внуково,2,5,31.00,14.61,10.00,2021,...,unknown,unknown,Нет,Вторичка,unknown,1,0.400000,31.00,0.471290,0.322581
2,20206500,55.810466,37.624247,Алексеевская,34,37,28.50,21.40,7.62,2024,...,Монолитно-кирпичный,unknown,unknown,Новостройка,КОРТРОС,1,0.918919,28.50,0.750877,0.267368
3,17119620,55.706597,37.632285,Тульская,6,20,38.82,10.60,18.60,2024,...,Монолитный,unknown,unknown,Новостройка,unknown,1,0.300000,38.82,0.273055,0.479134
4,7550000,55.551456,37.339499,Филатов луг,2,5,30.00,15.00,9.00,2021,...,unknown,unknown,Нет,Вторичка,unknown,1,0.400000,30.00,0.500000,0.300000


In [3]:
with open("./models/model.pkl", "rb") as f:
    model = pickle.load(f)

In [4]:
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['geo_lat', 'geo_lng',
                                                   'floor', 'floor_count',
                                                   'square', 'living_square',
                                                   'kitchen_square', 'year',
                                                   'ceiling_height',
                                                   'room_count', 'floor_ratio',
                                                   'room_ratio', 'living_ratio',
                                                   'kitchen_ratio']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='igno...
                                              LGBMRegressor(colsample_bytree=0.535,
                                                            learning_rate=0.13473543057016707,
                                                            max_depth=12,
                                                            min_child_samples=1,
                                                            n_estimators=501,
                                                            num_leaves=43,
                                                            random_state=42,
                                                            reg_alpha=7.290351325928806,
                                                            reg_lambda=7.472804907720953,
                                                            scale_pos_weight=1.7,
                                                            subsample=0.75,
                                                            verbosity=-1)),
                                             ('cat',
                                              <catboost.core.CatBoostRegressor object at 0x777b9bc40500>)],
                                 n_jobs=-1, weights=(0.6, 0.2, 0.2)))])

In [5]:
y = df_cian["price"]

X_train, X_test, y_train, y_test = train_test_split(
    df_cian,
    y,
    test_size=0.1,
    random_state=42,
)

y_train = X_train["price"]
y_test = X_test["price"]


pred = model.predict(X_test.drop(columns=["price"]))
print(
    f"rooms all: MAE = {mean_absolute_error(y_test, pred)}, R2 = {r2_score(y_test, pred)}"
)


for i in range(1, 6 + 1):

    X = X_test[X_test["room_count"] == i]
    y = X["price"]
    X = X.drop(columns=["price"])

    pred = model.predict(X)
    print(
        f"rooms {i}: MAE = {mean_absolute_error(y, pred)}, R2 = {r2_score(y, pred)}"
    )

rooms all: MAE = 14353670.12898465, R2 = 0.9313597347172033
rooms 1: MAE = 1830411.2104753833, R2 = 0.7672682258721988
rooms 2: MAE = 3950169.131832248, R2 = 0.9050926064947961
rooms 3: MAE = 8857483.639621073, R2 = 0.8873503448151152
rooms 4: MAE = 11112475.112572271, R2 = 0.839397674552991
rooms 5: MAE = 20468987.444081843, R2 = 0.9411079121301286
rooms 6: MAE = 55938895.28817221, R2 = 0.8448991128503083
